# Import Library

In [65]:
import seaborn as sns
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import seaborn as sns
from xgboost import XGBClassifier
#from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer,accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss
import warnings
warnings.filterwarnings("ignore")

# Load Dataset

In [66]:
data=pd.read_csv('training.csv')
test=pd.read_csv('testing.csv',sep=';')
print(data.shape,test.shape)

(16625, 16) (16625, 15)


# Data Preprocessing

In [67]:
data['MARRIAGE']=pd.Categorical(data['MARRIAGE'])
data['SEX']=pd.Categorical(data['SEX'])

test['MARRIAGE']=pd.Categorical(test['MARRIAGE'])
test['SEX']=pd.Categorical(test['SEX'])


# Get the Variable

In [68]:
y=data['TARGET']
data=data.drop('TARGET',1)
datagabung=pd.concat([data,test],0)
datagabung_drop=datagabung.drop(['ID'],1)
newdata=datagabung_drop[['PAY_1','PAY_2','PAY_3','LIMIT_BAL','BILL_AMT3','PAY_AMT3']]
newdata.head()


,PAY_1,PAY_2,PAY_3,LIMIT_BAL,BILL_AMT3,PAY_AMT3
0,0,0,0,30000.0,23881.0,1287.0
1,1,2,0,20000.0,17203.0,1500.0
2,0,0,0,20000.0,17947.0,1178.0
3,0,0,0,20000.0,3372.0,0.0
4,0,0,0,170000.0,110163.0,3760.0


## Split Train and Validation Dataset

In [71]:
df_train = Xgabung[:len(data)]
df_test = Xgabung[len(data):]

In [72]:
#df_train.head()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train,y,test_size=0.3)

## Model Comparison

In [73]:
# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
## collect all model in one list
all_model = [DecisionTreeClassifier,
            LogisticRegression,
             KNeighborsClassifier,
             GaussianNB,
#            SVC,
            LinearSVC,
            RandomForestClassifier,
            GradientBoostingClassifier,
            ExtraTreesClassifier]

desc_model = ['DecisionTreeClassifier',
            'LogisticRegression',
             'KNeighborsClassifier',
             'GaussianNB',
#            'SVC',
            'LinearSVC',
            'RandomForestClassifier',
            'GradientBoostingClassifier',
            'ExtraTreesClassifier']


all_model

[sklearn.tree.tree.DecisionTreeClassifier,
 sklearn.linear_model.logistic.LogisticRegression,
 sklearn.neighbors.classification.KNeighborsClassifier,
 sklearn.naive_bayes.GaussianNB,
 sklearn.svm.classes.LinearSVC,
 sklearn.ensemble.forest.RandomForestClassifier,
 sklearn.ensemble.gradient_boosting.GradientBoostingClassifier,
 sklearn.ensemble.forest.ExtraTreesClassifier]

In [74]:
## loop for all model

datatr = []
datasc = []
Recall =[]
Precision =[]
auc =[]

for i in all_model:
    model = i()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    datatr.append(model.score( X_train , y_train ))
    datasc.append(model.score( X_test , y_test ))
    Recall.append(recall_score(y_test,y_pred))
    Precision.append(precision_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
   # print(i, model.score( X_train , y_train ) , model.score( X_test , y_test ))

## Rank the Modelling Score

In [75]:
## compare model each other
data1 = pd.DataFrame()
data1['model'] = desc_model
data1['Accuracy training'] = datatr
data1['Accuracy test'] = datasc
data1['Precision'] = Precision
data1['Recall']= Recall
data1['AUC']=auc
data1['gap'] = abs(data1['Accuracy training'] - data1['Accuracy test'])
data1.sort_values(by='Accuracy test',ascending=False)

,model,Accuracy training,Accuracy test,Precision,Recall,AUC,gap
6,GradientBoostingClassifier,0.824611,0.817362,0.819953,0.696241,0.797168,0.007249
5,RandomForestClassifier,0.964939,0.799719,0.786207,0.685714,0.780712,0.165220
7,ExtraTreesClassifier,0.982728,0.793103,0.767947,0.691729,0.776202,0.189624
0,DecisionTreeClassifier,0.982728,0.765437,0.712082,0.694236,0.753566,0.217290
2,KNeighborsClassifier,0.816104,0.731155,0.679473,0.620551,0.712715,0.084949
1,LogisticRegression,0.616740,0.614274,0.986301,0.036090,0.517878,0.002465
4,LinearSVC,0.601358,0.600040,0.000000,0.000000,0.500000,0.001318
3,GaussianNB,0.597319,0.585806,0.487253,0.680201,0.601544,0.011513


We Choose Gradient Boosting Classifier because it has the highest AUC Score

# Hyperparameter Tuning for Gradient Boosting

In [76]:
xgboost=GradientBoostingClassifier(learning_rate=0.01,n_estimators=750,max_depth=7,min_samples_split=2)
xgboost.fit(X_train,y_train)
%time
y_pred=model.predict(X_test)

Wall time: 670 µs


# Fit Random Forest

In [77]:
RF=RandomForestClassifier(n_estimators=750,max_depth=7,min_samples_split=2)
RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# 

In [78]:
import pickle
import requests
import json
pickle.dump(xgboost, open('xgboost.pkl','wb'))
pickle.dump(RF, open('RF.pkl','wb'))
pickle.dump(RF, open('RF.pkl','wb'))

In [79]:
model_xgb=pickle.load(open('xgboost.pkl','rb'))
model_rf=pickle.load(open('RF.pkl','rb'))

In [80]:
xgb_y_pred=model_xgb.predict(X_test)
print(classification_report(y_test,xgb_y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      2993
           1       0.82      0.72      0.77      1995

   micro avg       0.83      0.83      0.83      4988
   macro avg       0.82      0.81      0.81      4988
weighted avg       0.83      0.83      0.82      4988



In [81]:
RF_y_pred=model_rf.predict(X_test)
print(classification_report(y_test,RF_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85      2993
           1       0.82      0.69      0.75      1995

   micro avg       0.82      0.82      0.82      4988
   macro avg       0.82      0.80      0.80      4988
weighted avg       0.82      0.82      0.81      4988

